In [54]:
import pandas as pd
import requests
import re

In [55]:
practices_path = "../../master/procurement_practices.csv"
df_practices = pd.read_csv(practices_path)

In [56]:
def get_status_code(url):
  try:
    response = requests.head(url, allow_redirects=True)
    return int(response.status_code)
  except:
    return None

In [57]:
df_practices["source_url"] = df_practices['source_or_evidence'].apply(lambda x: re.findall(r'(https?://\S+|www\.\S+|\S+\.(com|org|net))', x) if pd.notna(x) else x)
df_practices["source_url"] = df_practices["source_url"].apply(
    lambda x: [url[0].replace('href=', '')
               .replace('"', '')
               .replace("'", '')
               .replace('(', '')
               .replace(')', '')
               .replace('</strong>', '')
               .split('>')[0] if '>' in url[0] else url[0]
            for url in x] if isinstance(x, list) else x
)
df_practices["source_url"] = df_practices["source_url"].apply(
    lambda x: [url.replace('(', '').replace(')', '') for url in x] if isinstance(x, list) else x
)

# Create a new dataframe with the cleaned URLs
extracted_links = df_practices.explode('source_url')[['Practice', 'source_url']].dropna().reset_index(drop=True)

In [58]:
extracted_links['validate_url'] = extracted_links['source_url'].apply(get_status_code)
extracted_links.to_csv("../../master/procurement_source_urls.csv", index=False)